# Tutorial dla Tweepy

## Podstawowe możliwosci

### Instalacja i uwierzytelnienie

Pobieramy najnowszą wersję.

In [ ]:
!pip install tweepy --upgrade

Aby się uwierzytelnić używamy naszego tokenu z developer.twitter.com

Poniższy został przyznany w ramach grantu akademickiego, pozwala na pobranie 10 milionow Twittow miesięcznie i korzystać z wszystkich fukcjonalnosć Twittera API (przede wszystkim z wyszukiwania Twittow starszych niż tydzień).

In [12]:
academic_bearer = ""

In [ ]:
import tweepy

In [ ]:
client = tweepy.Client(academic_bearer)

### Wyszukiwanie Tweetow

W przypadku normalnego dostępu posługujemy się funkcją, ktora znajduje tweety z ostatniego tygodnia:

In [ ]:
query = 'from:JHaszczynski -is:retweet'
tweet_fields = ['created_at']
max_result = 10

tweets = client.search_recent_tweets(query=query, tweet_fields=tweet_fields, max_results=max_result)

for tweet in tweets.data:
    print(tweet.text)
    print(tweet.created_at)
    print("\n\n")


Mając dostęp do grantu akademickiego możemy brać tweety starsze niż tydzień korzystając z funkcji ```search_all_tweets```, maksymalna liczba zwracanych tweetow to 500

In [ ]:
query = 'from:juliaioffe -is:retweet'
tweet_fields = ['created_at']
max_result = 250

tweets = client.search_all_tweets(query=query, tweet_fields=tweet_fields, max_results=max_result)

for tweet in tweets.data:
    print(tweet.text)
    print(tweet.created_at)
    print("\n\n")

### Zapis wielu tweetow do pliku

In [ ]:
query = 'from:juliaioffe -is:retweet'
tweet_fields = ['created_at']
max_result = 500
number_of_wanted_tweets = 1000

file_name = "many_tweets.tsv"
with open(file_name, 'a+') as filehandle:
    for tweet in tweepy.Paginator(client.search_all_tweets, query=query, tweet_fields=tweet_fields, max_results=max_result).flatten(limit=number_of_wanted_tweets):
        filehandle.write(f"{tweet.text}\t{tweet.created_at}\n")

### Użycie query z rożnymi znacznikami

Lista znajduje się [tutaj](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query)

Poniżej kilka przykładow:



In [ ]:
query = '#Ukraine lang:en place_country:RU'
tweet_fields = ['created_at']
max_result = 50

#tweets = client.search_all_tweets(query=query, tweet_fields=tweet_fields, max_results=max_result)

#for tweet in tweets.data:
#    print(tweet.text)
#    print(tweet.created_at)
#    print("\n\n")

### Oczyszczania tweetow

Informacje o wspomnianych użytkownikach oraz o retweetach znajduje się także w Twitter fields. 

In [ ]:
import re

In [ ]:
RETWEETED_REGEX = r"^RT @([^ ]+):"
USER_REGEX = r"@([^ ]+)"
URL_REGEX = r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"


In [ ]:
def clean_tweet(tweet):
    is_retweet = False if re.match(RETWEETED_REGEX, tweet) is None else True
    di = {"is_retweet": is_retweet,
          "retweeted_user": re.findall(RETWEETED_REGEX, tweet)[0] if is_retweet else "",
          "links": re.findall(URL_REGEX, tweet)}
    tweet = re.sub(RETWEETED_REGEX, "", tweet)
    di["mentioned_users"] = re.findall(USER_REGEX, tweet)
    tweet = re.sub(USER_REGEX, "", tweet)
    tweet = re.sub(URL_REGEX, "", tweet)
    di["cleaned_tweet"] = tweet.strip()
    return di

### Użytkownicy

Jak znaleźć użytkownikow danych tweetow

In [ ]:
query = 'Ukraina lang:pl place_country:PL'
tweet_fields = ['created_at']
max_result = 40
expansion = "author_id"

tweets = client.search_all_tweets(query=query, tweet_fields=tweet_fields, expansions=expansion, max_results=max_result)

users = []

for tweet in tweets.data:
    tweet_dict = clean_tweet(tweet.text)
    #print(tweet.author_id)
    user = client.get_user(id=tweet.author_id, user_fields=["username"])
    print(user.data.name)
    print(user.data.username)  #Username jest najważniejsze
    print(tweet_dict["cleaned_tweet"])
    users.append(user.data.username)
    print("\n\n")

Jesli chcemy wypisać tweety konkretnych użytkownikow

In [ ]:
users_to_check_list = ["AgnieszkaO2GA",
                          "BeataDbrowska9",
                          "Albert301271",
                          "RenaTa99609630",
                          "DemonDakki",
                          "PoPrawnik2",
                          "MarcJachacz"]

users_to_check_ids = [client.get_user(username=user).data.id for user in users_to_check_list]

In [ ]:
print(users_to_check_ids)

In [ ]:
tweet_fields = ['created_at']
max_result = 10
for user_id, user_name in zip(users_to_check_ids, users_to_check_list):
  print(user_name)
  tweets = client.get_users_tweets(user_id, max_results=max_result, tweet_fields=tweet_fields)
  for tweet in tweets.data:
    print(tweet.text)
    print(tweet.created_at)
  print("END OF THIS USER \n\n\n")

### Przykładowa klasa do sprawdzania podstawowych parametrow związanych z użytkownikiem:

In [ ]:
!pip install matplotlib
!pip install pandas
!pip install wordcloud


In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def create_word_cloud(lista):
  comment_words = ''
  stopwords = set(STOPWORDS)
  
  for item in lista:
      
      tokens = item["cleaned_tweet"].split()
      tokens = [token.lower() for token in tokens]
      comment_words += " ".join(tokens)+" "
  
  wordcloud = WordCloud(width = 800, height = 800,
                  background_color ='white',
                  stopwords = stopwords,
                  min_font_size = 10).generate(comment_words)
  
  plt.figure(figsize = (8, 8), facecolor = None)
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout(pad = 0)
  
  plt.show()

In [ ]:
from datetime import datetime
from datetime import timedelta

def get_old_data(last_days_num):
       now = datetime.now()
       then = now - timedelta(days=last_days_num)
       month = str(then.month).zfill(2)
       day = str(then.day).zfill(2)
       start_date = f"{then.year}-{month}-{day}T00:00:00Z"
       return start_date

In [ ]:
class UserStats(object):
    def __init__(self, user):
        self.client = client
        self.username = user
        user = self.client.get_user(username=user, user_fields=["created_at", "public_metrics", "verified"])
        self.id = user.data.id
        self.created = user.data.created_at
        self.followers_number = user.data.public_metrics["followers_count"]
        self.followed_number = user.data.public_metrics["following_count"]
        self.tweet_number = user.data.public_metrics["tweet_count"]
        self.if_verified = user.data.verified

    def get_tweets(self, previous_days=30):
        self.previous_days = previous_days
        old_data = get_old_data(previous_days)
        query = f"from:{self.username}"
        self.last_tweets = self.client.search_all_tweets(query, start_time=old_data, max_results=300, tweet_fields=["public_metrics"])
        self.processed_tweets = [clean_tweet(tweet.text) for tweet in self.last_tweets.data]

    def calculate_popularity(self):
        self.popularity_score = sum([tt.public_metrics["retweet_count"] for tt in self.last_tweets.data]) + \
                                sum([tt.public_metrics["like_count"] for tt in self.last_tweets.data])
        self.reach_score = self.followers_number - self.followed_number

    def get_followers(self, max_results=10):
        followers = self.client.get_API().get_users_followers(self.id, max_results=max_results)
        return [follower.username for follower in followers.data]

    def print_basic_stats(self):
        print(f"Popularity score: {self.popularity_score}")
        print(f"Reach score: {self.reach_score}")


In [ ]:
    joe = UserStats(user="JoeBiden")
    joe.get_tweets(previous_days=60)
    joe.calculate_popularity()
    joe.print_basic_stats()
    list_of_followers = joe.get_followers()
    for follower in list_of_followers:
        print(follower)

In [ ]:
create_word_cloud(joe.processed_tweets)

### Zadania

1. Sprobowac pisac własne zapytania (query) korzystając z instrukcji na stronie dokumentacji
2. Moża poszukać roznego rodzaju propagandystow, stworzyć dla nich wordcloud, sprawdzić ich followersow itd. 
3. Stworzyć własną metrykę popularnosci np. srednia ilosc retweetow na dzień albo na pojedynczego tweeta - mozna wykorzystac te klasę